## Gorilla assessment - David Meulenbeld - Datagrow

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime
import timeit

forecast_df = pd.read_excel('data.xlsx', sheet_name=0)
meter_list_df = pd.read_excel('data.xlsx', sheet_name=1)
rates_df = pd.read_excel('data.xlsx', sheet_name=2)

### 1

Steps:
1. Get max annual quantity to fill NaN values
2. Merge Meter list and rates
3. Get correct rate for annual usage
4. Sort dates neccessary for merge_asof
5. Calculate cost. KWH * Rate / 100 to convert to pounds
6. Groupby and sum cost

In [2]:
max_annual_quantity = meter_list_df['Annual Quantity (kWh)'].max() + 1

In [3]:
meters_rates_df = pd.merge(meter_list_df, rates_df, how="inner", on="Exit Zone")

In [4]:
meters_rates_df['Annual Quantity (Max)'] = meters_rates_df['Annual Quantity (Max)'].fillna(max_annual_quantity)

In [5]:
meters_match_aq_df = meters_rates_df.loc[(meters_rates_df['Annual Quantity (kWh)'] > meters_rates_df['Annual Quantity (Min)']) & (meters_rates_df['Annual Quantity (kWh)'] < meters_rates_df['Annual Quantity (Max)'])]

In [6]:
meters_match_aq_df = meters_match_aq_df.sort_values(by=['Date'])

In [7]:
forecast_with_rates = pd.merge_asof(forecast_df, meters_match_aq_df, on='Date', by='Meter ID')

In [8]:
forecast_with_rates['Total Cost £'] = (forecast_with_rates['kWh'] * forecast_with_rates['Rate (p/kWh)']) / 100

In [9]:
forecast_with_rates.groupby(['Meter ID']).sum()[['kWh', 'Total Cost £']].round(decimals=2).rename(columns={"kWh": "Total Estimated Consumption (kWh)"})

,Total Estimated Consumption (kWh),Total Cost £
Meter ID,,
10588707,122450.0,28.50
10626610,303558.0,88.24
1000000603,24674.0,8.27


### 2
Steps:
1. Get all possible exit zones
2. Start loop for given list size
3. Randomize int to use as index for exit zone list
4. Randomize Meter id and Annual quantity
5. Pick exit zone

In [10]:
def generate_meters(meter_list_size):
    meters = []
    possible_exit_zones = rates_df['Exit Zone'].unique()
    for i in range(0, meter_list_size):
        number_exit_zone = random.randint(0, (len(possible_exit_zones)-1))
        meter_id = random.randint(1000, 1000000)
        annual_quantity = random.randint(0,10000000)
        exit_zone = possible_exit_zones[number_exit_zone]
        meters.append([meter_id, exit_zone, annual_quantity ])
    
    return meters

In [11]:
generated_meter_list = generate_meters(5)
generated_meter_list

[[938262, 'NW1', 871452],
 [119457, 'SW3', 4097163],
 [128101, 'SC2', 4595497],
 [234261, 'NT1', 3766850],
 [953960, 'NT1', 5195853]]

### 3
steps:
1. Create date range from given start and end date
2. For every day and every meter in given meter list randomize usage

In [12]:
def generate_consumption(meter_list, start_date, end_date):
    consumption = []
    date_range = pd.date_range(start_date, end_date)
    for date in date_range:
        for meter in meter_list:
            kwh = random.uniform(0,500)
            consumption.append([date, meter[0], kwh])
            
    return consumption

In [13]:
generated_consumption = generate_consumption(generated_meter_list, '01-01-2020', '01-01-2024')
generated_consumption

[[Timestamp('2020-01-01 00:00:00', freq='D'), 938262, 483.31487166159206],
 [Timestamp('2020-01-01 00:00:00', freq='D'), 119457, 424.21004224953293],
 [Timestamp('2020-01-01 00:00:00', freq='D'), 128101, 163.74993822004947],
 [Timestamp('2020-01-01 00:00:00', freq='D'), 234261, 258.7419683503652],
 [Timestamp('2020-01-01 00:00:00', freq='D'), 953960, 304.5164068326317],
 [Timestamp('2020-01-02 00:00:00', freq='D'), 938262, 300.22726930059287],
 [Timestamp('2020-01-02 00:00:00', freq='D'), 119457, 236.60929824996978],
 [Timestamp('2020-01-02 00:00:00', freq='D'), 128101, 107.04914517502439],
 [Timestamp('2020-01-02 00:00:00', freq='D'), 234261, 497.69492881312914],
 [Timestamp('2020-01-02 00:00:00', freq='D'), 953960, 282.75749907278185],
 [Timestamp('2020-01-03 00:00:00', freq='D'), 938262, 471.96679658200145],
 [Timestamp('2020-01-03 00:00:00', freq='D'), 119457, 33.659800325595846],
 [Timestamp('2020-01-03 00:00:00', freq='D'), 128101, 21.08715686603685],
 [Timestamp('2020-01-03 00:0

### 4

Same process as the first step. Made Pandas dataframes from input list.

Tested different meter list sizes and a longer period.

In [14]:
def generate_cost_table(generated_meter_list, generated_consumption, rates_df):
    forecast_df = pd.DataFrame(generated_consumption, columns=['Date','Meter ID', 'kWh'])
    meter_df = pd.DataFrame(generated_meter_list, columns=['Meter ID', 'Exit Zone', 'Annual Quantity (kWh)'])

    max_annual_quantity = meter_df['Annual Quantity (kWh)'].max() + 1

    meters_rates_df = pd.merge(meter_df, rates_df, how="inner", on="Exit Zone")

    meters_rates_df['Annual Quantity (Max)'] = meters_rates_df['Annual Quantity (Max)'].fillna(max_annual_quantity)

    meters_match_aq_df = meters_rates_df.loc[(meters_rates_df['Annual Quantity (kWh)'] > meters_rates_df['Annual Quantity (Min)']) & (meters_rates_df['Annual Quantity (kWh)'] < meters_rates_df['Annual Quantity (Max)'])]
    meters_match_aq_df = meters_match_aq_df.sort_values(by=['Date'])
    forecast_with_rates = pd.merge_asof(forecast_df, meters_match_aq_df, on='Date', by='Meter ID')
    forecast_with_rates['Total Cost £'] = (forecast_with_rates['kWh'] * forecast_with_rates['Rate (p/kWh)']) / 100
    
    return forecast_with_rates.groupby(['Meter ID']).sum()[['kWh', 'Total Cost £']].round(decimals=2).rename(columns={"kWh": "Total Estimated Consumption (kWh)"})

In [15]:
generate_cost_table(generated_meter_list, generated_consumption, rates_df)

,Total Estimated Consumption (kWh),Total Cost £
Meter ID,,
119457,376210.47,1250.26
128101,360470.54,841.32
234261,360797.82,679.41
938262,374503.57,909.54
953960,369617.34,688.16


In [16]:
%timeit generate_cost_table(generated_meter_list, generated_consumption, rates_df)

9.49 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
meter_list_size_scale = [50, 500, 5000, 50000, 500000]
for x in meter_list_size_scale:
    %timeit generate_cost_table(generate_meters(x), generate_consumption(generated_meter_list, '01-01-2020', '01-01-2030'), rates_df)

23.9 ms ± 286 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
26.4 ms ± 221 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
52.3 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
307 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.41 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 5 

The time the calculations took is well within bounds. For 500000 meters the calculation took less than 5 seconds.

There are two aspects to improving the performance of such calculations. 
1. The method of calculation, for example vectorization or not.
2. The size of the data for the calculation, for example we can remove all rates from the rate table that have a exit zone that is not in the meter list. This minimizes the size of the data.

In conclusion: I would search for ways to minimize the data and test different methods and time these.

